In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_excel('/content/drive/MyDrive/NBS_RI/Cashtags_2017-2023.xlsx')

In [ ]:
# Filter tweets mentioning Prudential
stock_tweets = df[df['tweet'].str.contains('Prudential|$PRU', case=False, na=False)]

#Renaming 'created_at' to 'Date'
stock_tweets.rename(columns={'created_at': 'Date'}, inplace=True)

stock_tweets['Date'] = pd.to_datetime(stock_tweets['Date'])

#Extract date part
stock_tweets['Date'] = stock_tweets['Date'].dt.date

<ipython-input-4-7b022606555b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_tweets.rename(columns={'created_at': 'Date'}, inplace=True)
<ipython-input-4-7b022606555b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_tweets['Date'] = pd.to_datetime(stock_tweets['Date'])
<ipython-input-4-7b022606555b>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/index

In [ ]:
# Dropping unnecesary columns

stock_tweets.drop(['author id','id','lang'], axis=1, inplace=True)

<ipython-input-5-f07e51a1f46c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_tweets.drop(['author id','id','lang'], axis=1, inplace=True)


In [ ]:
import re
import string
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Preprocess tweet text
def preprocess_tweet(tweet):
    # Convert to lowercase
    tweet = tweet.lower()

    # Remove special characters
    tweet = re.sub(r'[^\w\s]', '', tweet)

    # Remove URLs
    tweet = re.sub(r'http\S+', '', tweet)
    tweet = re.sub(r'www\S+', '', tweet)

    # Remove hashtags and user mentions
    tweet = re.sub(r'#\w+', '', tweet)
    tweet = re.sub(r'@\w+', '', tweet)

    # Remove punctuation
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))

    # Tokenization
    tokens = word_tokenize(tweet)

    # Remove stop words
    tokens = [token for token in tokens if token not in stopwords.words('english')]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return ' '.join(tokens)

# Apply preprocessing
stock_tweets['cleaned_tweet'] = [preprocess_tweet(tweet) for tweet in stock_tweets['tweet']]

<ipython-input-7-b465d6e61760>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_tweets['cleaned_tweet'] = [preprocess_tweet(tweet) for tweet in stock_tweets['tweet']]


In [ ]:
#Removing duplicates
stock_tweets.drop_duplicates(subset=['cleaned_tweet'], inplace = True)

<ipython-input-8-e58614a5f446>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_tweets.drop_duplicates(subset=['cleaned_tweet'], inplace = True)


In [ ]:
stock_tweets.drop(columns=['tweet'],inplace= True)

<ipython-input-9-612f487d092f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_tweets.drop(columns=['tweet'],inplace= True)


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [ ]:
! pip install -q vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 1.8 MB/s eta 0:00:00


In [ ]:
import torch
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
sid = SentimentIntensityAnalyzer()

In [ ]:
def get_sentiment_scores_bert(tweet):
    inputs = tokenizer(tweet, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1).detach().cpu().numpy()[0]
    return probabilities

In [ ]:
def get_mean_sentiment_scores(tweet):
    bert_scores = get_sentiment_scores_bert(tweet)

    vader_scores = sid.polarity_scores(tweet)

    bert_negative = bert_scores[0]
    bert_neutral = bert_scores[1]
    bert_positive = bert_scores[2]

    vader_negative = vader_scores['neg']
    vader_neutral = vader_scores['neu']
    vader_positive = vader_scores['pos']

    mean_negative = (bert_negative + vader_negative) / 2
    mean_neutral = (bert_neutral + vader_neutral) / 2
    mean_positive = (bert_positive + vader_positive) / 2

    return mean_negative, mean_neutral, mean_positive

In [ ]:
def combined_sentiment_score(row):
    positive_score = row['positive']
    negative_score = row['negative']
    neutral_score = row['neutral']

    combined_score = positive_score - negative_score

    return combined_score

In [ ]:
stock_tweets

,Date,cleaned_tweet
8,2017-01-01,henderson group plc 4519000 stake prudential f...
29,2017-01-02,prudential financial inc pru rating lowered ho...
42,2017-01-02,king luther capital management corp sell 7270 ...
49,2017-01-02,prudentialfinancial inc cut position ag mortga...
52,2017-01-02,prudentialfinancial inc given buy rating sandl...
...,...,...
393764,2023-05-31,pru prudentials cfo quits code conduct investi...
395104,2023-05-31,pru prudentials cfo quits code conduct investi...
395750,2023-05-31,prudential plc pru dipped 60 533 trading 11k
395762,2023-05-31,prudential plc pru dipped 675 6 trading 11k


In [ ]:
# Apply the function to each row to get the neutral, negative and positive sentiment score
sentiment_scores = stock_tweets['cleaned_tweet'].apply(get_mean_sentiment_scores)

stock_tweets[['negative', 'neutral', 'positive']] = pd.DataFrame(sentiment_scores.tolist(), index=stock_tweets.index)

# Apply the function to each row to get the combined sentiment score
stock_tweets['combined_score'] = stock_tweets.apply(combined_sentiment_score, axis=1)

<ipython-input-18-56d9e3980ef4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_tweets[['negative', 'neutral', 'positive']] = pd.DataFrame(sentiment_scores.tolist(), index=stock_tweets.index)
<ipython-input-18-56d9e3980ef4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_tweets[['negative', 'neutral', 'positive']] = pd.DataFrame(sentiment_scores.tolist(), index=stock_tweets.index)
<ipython-input-18-56d9e3980ef4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [ ]:
stock_tweets

,Date,cleaned_tweet,negative,neutral,positive,combined_score
8,2017-01-01,henderson group plc 4519000 stake prudential f...,4.999877e-01,0.500000,1.197373e-05,-4.999757e-01
29,2017-01-02,prudential financial inc pru rating lowered ho...,1.191234e-01,0.436275,4.446021e-01,3.254787e-01
42,2017-01-02,king luther capital management corp sell 7270 ...,4.987537e-01,0.416514,8.473225e-02,-4.140214e-01
49,2017-01-02,prudentialfinancial inc cut position ag mortga...,5.678242e-01,0.324505,1.071705e-01,-4.606537e-01
52,2017-01-02,prudentialfinancial inc given buy rating sandl...,4.809756e-08,1.000000,3.913504e-08,-8.962516e-09
...,...,...,...,...,...,...
393764,2023-05-31,pru prudentials cfo quits code conduct investi...,4.941644e-01,0.500019,5.816103e-03,-4.883483e-01
395104,2023-05-31,pru prudentials cfo quits code conduct investi...,4.386074e-01,0.500044,6.134884e-02,-3.772586e-01
395750,2023-05-31,prudential plc pru dipped 60 533 trading 11k,4.611386e-01,0.500010,3.885172e-02,-4.222869e-01
395762,2023-05-31,prudential plc pru dipped 675 6 trading 11k,7.856139e-02,0.500038,4.214004e-01,3.428390e-01


In [ ]:
stock_tweets.drop(columns=['cleaned_tweet'], inplace=True)

<ipython-input-20-a714201e105a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_tweets.drop(columns=['cleaned_tweet'], inplace=True)


In [ ]:
stock_tweets

,Date,negative,neutral,positive,combined_score
8,2017-01-01,4.999877e-01,0.500000,1.197373e-05,-4.999757e-01
29,2017-01-02,1.191234e-01,0.436275,4.446021e-01,3.254787e-01
42,2017-01-02,4.987537e-01,0.416514,8.473225e-02,-4.140214e-01
49,2017-01-02,5.678242e-01,0.324505,1.071705e-01,-4.606537e-01
52,2017-01-02,4.809756e-08,1.000000,3.913504e-08,-8.962516e-09
...,...,...,...,...,...
393764,2023-05-31,4.941644e-01,0.500019,5.816103e-03,-4.883483e-01
395104,2023-05-31,4.386074e-01,0.500044,6.134884e-02,-3.772586e-01
395750,2023-05-31,4.611386e-01,0.500010,3.885172e-02,-4.222869e-01
395762,2023-05-31,7.856139e-02,0.500038,4.214004e-01,3.428390e-01


In [ ]:
# Group by 'Date' and calculate the mean for 'negative', 'neutral', and 'positive'
stock_tweets_sentiment = stock_tweets.groupby('Date').agg({
    'negative': 'mean',
    'neutral': 'mean',
    'positive': 'mean',
    'combined_score': 'mean'
}).reset_index()

In [ ]:
stock_tweets_sentiment

,Date,negative,neutral,positive,combined_score
0,2017-01-01,0.499988,0.500000,0.000012,-0.499976
1,2017-01-02,0.296425,0.544323,0.159126,-0.137299
2,2017-01-03,0.441265,0.489222,0.069512,-0.371753
3,2017-01-04,0.403945,0.509964,0.086091,-0.317854
4,2017-01-05,0.374976,0.423502,0.201521,-0.173455
...,...,...,...,...,...
1975,2023-05-25,0.524454,0.440038,0.035508,-0.488945
1976,2023-05-26,0.387832,0.502415,0.109754,-0.278078
1977,2023-05-28,0.615427,0.384504,0.000070,-0.615357
1978,2023-05-30,0.250000,0.698750,0.051250,-0.198750


In [ ]:
stock_tweets_sentiment.to_excel('stock_tweets_sentiment(VADER+BERT).xlsx', index = False)